In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim) 
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[7],5
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'o3': 'atmpro'}","{'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro'}"
nv,-,1000
tsfc,294,294


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.039602                    1 -0.031063
0.000750        2 -0.043982                    2 -0.034513
0.001052        3 -0.052429                    3 -0.041254
0.001476        4 -0.060531                    4 -0.047739
0.002070        5 -0.068301                    5 -0.054014
0.002904        6 -0.075751                    6 -0.060050
0.004074        7 -0.082859                    7 -0.065875
0.005714        8 -0.089619                    8 -0.071422
0.008015        9 -0.096038                    9 -0.076763
0.011243       10 -0.102139                   10 -0.081807
0.015771       11 -0.108066                   11 -0.086624
0.022122       12 -0.113837                   12 -0.091229
0.031031       13 -0.119616                   13 -0.095602
0.043528       14 -0.125348                   14 -0.099682
0.061057       15 -0.127273                   15 -0.100933
0.085645       16 -0.117989                   16 -0.094561
0.120136       17 -0.069534                   17 -0.057536
0.168516       18  0.033677                   18  0.026912
0.236378       19  0.202039                   19  0.171347
0.331549       20  0.447926                   20  0.391236
0.465100       21  0.784813                   21  0.703507
0.652400       22  1.232713                   22  1.124570
0.915100       23  1.754147                   23  1.624947
1.283650       24  2.016286                   24  1.929227
1.800600       25  1.808280                   25  1.795501
2.525700       26  1.449857                   26  1.444981
3.542800       27  1.072546                   27  1.063298
4.969550       28  0.753432                   28  0.761406
6.970850       29  0.530250                   29  0.561402
9.778100       30  0.359931                   30  0.404942
13.715850      31  0.187314                   31  0.228021
19.239350      32  0.018676                   32  0.042665
26.987250      33 -0.099576                   33 -0.092302
37.855300      34 -0.182464                   34 -0.191059
53.100050      35 -0.252031                   35 -0.277970
73.887500      36 -0.274543                   36 -0.314677
97.662500      37 -0.243610                   37 -0.283652
121.437500     38 -0.211238                   38 -0.244197
145.212500     39 -0.196616                   39 -0.221440
168.987500     40 -0.176079                   40 -0.191980
192.762500     41 -0.141177                   41 -0.150295
216.537500     42 -0.112040                   42 -0.117730
240.312500     43 -0.092085                   43 -0.095662
264.087500     44 -0.073675                   44 -0.075715
287.862500     45 -0.059101                   45 -0.060191
311.637500     46 -0.049318                   46 -0.049851
335.412500     47 -0.040926                   47 -0.041024
359.187500     48 -0.033095                   48 -0.032938
382.962500     49 -0.026275                   49 -0.025988
406.737500     50 -0.020322                   50 -0.019972
430.512500     51 -0.014774                   51 -0.014410
454.287500     52 -0.009622                   52 -0.009291
478.062500     53 -0.004843                   53 -0.004551
501.837500     54 -0.000419                   54 -0.000175
525.612500     55  0.003806                   55  0.003978
549.387500     56  0.007950                   56  0.008047
573.162500     57  0.012095                   57  0.012120
596.937500     58  0.016554                   58  0.016493
620.712500     59  0.021189                   59  0.021059
644.487500     60  0.026000                   60  0.025773
668.262500     61  0.031020                   61  0.030667
692.037500     62  0.036228                   62  0.035760
715.812500     63  0.041685                   63  0.041174
739.587500     64  0.047431                   64  0.046971
763.362500     65  0.053435                   65  0.053164
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug      fldg      fnetg       flug          fldg   
pressure    level                                                            
0.000000    1     -22.506210  0.000000 -22.506210 -22.123075 -5.323227e-08   
0.000624    2     -22.506215  0.000001 -22.506213 -22.123078  9.797291e-07   
0.000876    3     -22.506217  0.000002 -22.506215 -22.123079  1.449101e-06   
0.001229    4     -22.506220  0.000003 -22.506217 -22.123082  2.261489e-06   
0.001723    5     -22.506225  0.000005 -22.506220 -22.123086  3.628842e-06   
0.002417    6     -22.506234  0.000008 -22.506226 -22.123093  5.878008e-06   
0.003391    7     -22.506248  0.000014 -22.506235 -22.123103  9.518221e-06   
0.004757    8     -22.506270  0.000022 -22.506248 -22.123120  1.532920e-05   
0.006672    9     -22.506303  0.000035 -22.506268 -22.123145  2.450998e-05   
0.009359    10    -22.506354  0.000055 -22.506299 -22.123184  3.887977e-05   
0.013128    11    -22.506431  0.000086 -22.506345 -22.123243  6.121693e-05   
0.018415    12    -22.506547  0.000135 -22.506412 -22.123332  9.571908e-05   
0.025830    13    -22.506721  0.000209 -22.506512 -22.123465  1.487031e-04   
0.036232    14    -22.506982  0.000322 -22.506660 -22.123664  2.297047e-04   
0.050823    15    -22.507369  0.000493 -22.506876 -22.123959  3.530993e-04   
0.071291    16    -22.507953  0.000768 -22.507185 -22.124404  5.534130e-04   
0.100000    17    -22.508902  0.001316 -22.507586 -22.125130  9.571508e-04   
0.140271    18    -22.510436  0.002518 -22.507918 -22.126306  1.858759e-03   
0.196760    19    -22.512709  0.005017 -22.507692 -22.128044  3.777209e-03   
0.275997    20    -22.515773  0.009977 -22.505796 -22.130350  7.691714e-03   
0.387100    21    -22.519338  0.019437 -22.499901 -22.132910  1.540140e-02   
0.543100    22    -22.522250  0.036852 -22.485398 -22.134619  3.011157e-02   
0.761700    23    -22.521227  0.067750 -22.453477 -22.132564  5.717997e-02   
1.068500    24    -22.509968  0.120241 -22.389727 -22.121360  1.050372e-01   
1.498800    25    -22.484641  0.197688 -22.286953 -22.097487  1.795112e-01   
2.102400    26    -22.448713  0.291053 -22.157659 -22.063416  2.738328e-01   
2.949000    27    -22.404814  0.392555 -22.012259 -22.021247  3.765905e-01   
4.136600    28    -22.358259  0.496885 -21.861374 -21.975975  4.809181e-01   
5.802500    29    -22.317253  0.604560 -21.712693 -21.935630  5.908427e-01   
8.139200    30    -22.289358  0.723437 -21.565921 -21.907490  7.181137e-01   
11.417000   31    -22.285775  0.859608 -21.426167 -21.902477  8.703476e-01   
16.014700   32    -22.329393  1.005243 -21.324150 -21.943727  1.035797e+00   
22.464000   33    -22.453901  1.144019 -21.309882 -22.065877  1.190545e+00   
31.510500   34    -22.694494  1.277904 -21.416590 -22.308159  1.333904e+00   
44.200100   35    -23.105433  1.414568 -21.690864 -22.734736  1.473257e+00   
62.000000   36    -23.757392  1.535114 -22.222278 -23.441445  1.593800e+00   
85.775000   37    -24.614535  1.619056 -22.995479 -24.412756  1.678789e+00   
109.550000  38    -25.352119  1.670558 -23.681561 -25.264668  1.731764e+00   
133.325000  39    -25.988122  1.711647 -24.276475 -25.994535  1.773823e+00   
157.100000  40    -26.576264  1.746056 -24.830208 -26.653736  1.809314e+00   
180.875000  41    -27.104648  1.778546 -25.326102 -27.227550  1.842397e+00   
204.650000  42    -27.541578  1.817877 -25.723701 -27.688953  1.880478e+00   
228.425000  43    -27.907987  1.868745 -26.039243 -28.068300  1.928226e+00   
252.200000  44    -28.227869  1.929285 -26.298584 -28.394199  1.984682e+00   
275.975000  45    -28.500772  1.994697 -26.506075 -28.668395  2.045619e+00   
299.750000  46    -28.736041  2.063518 -26.672523 -28.902025  2.109714e+00   
323.525000  47    -28.949084  2.137666 -26.811418 -29.111510  2.178788e+00   
347.300000  48    -29.143390  2.216711 -26.926679 -29.300715  2.252445e+00   
371.075000  49    -29.319347  2.299463 -27.0198

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')